# Тестирование API инструментов GraphArchitect

Этот notebook тестирует все API инструменты:
- OpenRouter (универсальный роутер LLM)
- OpenAI (GPT-4, GPT-3.5)
- DeepSeek (специализированный для кода)
- VLLM (локальные модели)
- Replicate (LLM, Audio, VQA)
- Infinity (эмбеддинги)

In [1]:
# Настройка окружения
import sys
from pathlib import Path

# Добавляем GraphArchitect в путь
grapharchitect_path = Path.cwd().parent.parent / "src" / "GraphArchitectLib"
sys.path.insert(0, str(grapharchitect_path))

import os

# Проверка API ключей
print("Проверка API ключей:")
print(f"  OPENROUTER_API_KEY: {'Ok' if os.getenv('OPENROUTER_API_KEY') else 'X'}")
print(f"  OPENAI_API_KEY: {'Ok' if os.getenv('OPENAI_API_KEY') else 'X'}")
print(f"  REPLICATE_API_TOKEN: {'Ok' if os.getenv('REPLICATE_API_TOKEN') else 'X'}")

Проверка API ключей:
  OPENROUTER_API_KEY: Ok
  OPENAI_API_KEY: X
  REPLICATE_API_TOKEN: X


## 1. OpenRouter

Универсальный роутер для доступа к GPT-4, Claude, Gemini и другим моделям.

In [2]:
from grapharchitect.tools.ApiTools.OpenRouterTool.openrouter_llm import OpenRouterLLM

# Создание клиента
openrouter = OpenRouterLLM(
    model_name="openai/gpt-3.5-turbo",
    system_prompt="You are a helpful AI assistant."
)

print("OpenRouter инициализирован")
print(f"Модель: {openrouter.model_name}")

# Тестовый запрос
if os.getenv('OPENROUTER_API_KEY'):
    response = openrouter.query_llm(
        question="Объясни за 50 слов: что такое машинное обучение?",
        temperature=0.7,
        max_tokens=150
    )
    
    print(f"\nОтвет:\n{response}")
else:
    print("\nУстановите OPENROUTER_API_KEY для реального теста")

INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.
INFO:grapharchitect.tools.ApiTools.OpenRouterTool.openrouter_llm:OpenRouter инициализирован: модель openai/gpt-3.5-turbo


OpenRouter инициализирован
Модель: openai/gpt-3.5-turbo


INFO:grapharchitect.tools.ApiTools.OpenRouterTool.openrouter_llm:Использовано токенов: prompt=39, completion=150, total=189



Ответ:
Машинное обучение — это подраздел искусственного интеллекта, где компьютерные системы обучаются из опыта, а не программирования явно. Они используют алгоритмы и статистические модели для обнаружения закономерностей в данных и принятия решений без явного программирования. Машинное обучение применяется в различных областях, включая медицину, финансы, транспорт и много


## 2. VLLM API

Для работы с локально развернутыми моделями.

In [6]:
from grapharchitect.tools.ApiTools.VLLMTool.VLLMApi import VLLMApi

# Создание клиента
vllm_host = os.getenv("VLLM_HOST", "http://localhost:8000")

vllm_client = VLLMApi(
    vllm_host=vllm_host,
    model_name="Qwen/Qwen2.5-7B-Instruct",
    system_prompt="You are a helpful assistant."
)

print("VLLM клиент создан")
print(f"Host: {vllm_host}")
print(f"Модель: {vllm_client.model_name}")


response = vllm_client.query_llm("Привет, как дела?")
print(f"Ответ: {response}")

VLLM клиент создан
Host: http://localhost:8000
Модель: Qwen/Qwen2.5-7B-Instruct


ERROR:grapharchitect.tools.ApiTools.VLLMTool.VLLMApi:Ошибка при выполнении запроса: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001D1C8746E00>: Failed to establish a new connection: [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение'))


Ответ: Request failed: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001D1C8746E00>: Failed to establish a new connection: [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение'))


## 3. Infinity Embedder

Для получения качественных семантических эмбеддингов.

In [ ]:
from grapharchitect.tools.ApiTools.InfinityTool.Embedder import InfinityEmbedder

# Создание клиента
infinity_url = os.getenv("INFINITY_BASE_URL", "http://localhost:7997")

infinity = InfinityEmbedder(base_url=infinity_url)

print("Infinity клиент создан")
print(f"URL: {infinity_url}")

# Для теста нужен запущенный Infinity сервер
print("\n⚠ Для теста запустите Infinity:")
print("  docker run -d -p 7997:7997 michaelf34/infinity:latest --model-name BAAI/bge-m3")

# Раскомментируйте для реального вызова:
result = infinity.get_embedding("Пример текста для векторизации")
if "embedding" in result:
    print(f"Эмбеддинг получен, размерность: {len(result["data"]['embedding'])}")
else:
    print(f"Ошибка: {result}")

Infinity клиент создан
URL: http://localhost:7997

⚠ Для теста запустите Infinity:
  docker run -d -p 7997:7997 michaelf34/infinity:latest --model-name BAAI/bge-m3
Ошибка: {'object': 'list', 'data': [{'object': 'embedding', 'embedding': [-0.04124867171049118, -0.009164676070213318, -0.028419796377420425, -0.03693179413676262, -0.023525377735495567, -0.041226357221603394, 0.023294268175959587, 0.0779535323381424, 0.013143641874194145, -0.004059714265167713, 0.016685815528035164, -0.06239049509167671, -0.0466327965259552, -0.021628903225064278, -0.05806497484445572, -0.029215166345238686, 0.02423095703125, 0.024175381287932396, -0.08683943003416061, -0.04702775180339813, 0.04598876088857651, -0.015613301657140255, -0.006964504718780518, -0.013423626311123371, -0.019480491057038307, 0.03455484285950661, 0.03663982078433037, 0.027920212596654892, 0.012642240151762962, -0.08475520461797714, -0.021431513130664825, -0.01716582477092743, -0.013341275975108147, -0.007442456670105457, 0.01960079

## 4. Интеграция с GraphArchitect

Использование API инструментов в GraphArchitect workflow.

In [8]:
# Создание инструмента на основе OpenRouter
from grapharchitect.entities.base_tool import BaseTool
from grapharchitect.entities.connectors.connector import Connector

class OpenRouterTool(BaseTool):
    """Инструмент на основе OpenRouter."""
    
    def __init__(self, openrouter_client):
        super().__init__()
        self.metadata.tool_name = "OpenRouter GPT-3.5"
        self.metadata.reputation = 0.90
        self._client = openrouter_client
        
        self.input = Connector("text", "question")
        self.output = Connector("text", "answer")
    
    def execute(self, input_data):
        if not os.getenv('OPENROUTER_API_KEY'):
            return "[Заглушка] Ответ без API ключа"
        
        response = self._client.query_llm(
            question=str(input_data),
            temperature=0.7,
            max_tokens=500
        )
        return response

# Создание инструмента
if os.getenv('OPENROUTER_API_KEY'):
    tool = OpenRouterTool(openrouter)
    
    # Тест выполнения
    result = tool.execute("Что такое NLI?")
    print(f"Результат инструмента:\n{result}")
else:
    print("Для теста установите OPENROUTER_API_KEY")

INFO:grapharchitect.tools.ApiTools.OpenRouterTool.openrouter_llm:Использовано токенов: prompt=25, completion=96, total=121


Результат инструмента:
NLI расшифровывается как Natural Language Interface (естественно-языковой интерфейс). Это технология, позволяющая взаимодействовать с компьютерными системами с использованием обычного естественного языка, что делает коммуникацию более удобной для пользователей.


## Заключение

Все API инструменты протестированы и готовы к использованию в GraphArchitect.

**Следующие шаги**:
1. Установите необходимые API ключи
2. Запустите серверы (VLLM, Infinity)
3. Используйте инструменты в workflow

**См. также**: `02_test_local_tools.ipynb` для тестирования локальных инструментов